The story of Schrödinger's cat, a cat that through some quantum physics magic is simultaneously alive and dead, has become engrained in popular culture and many popular science articles. But as the physics has become popularized, misconceptions have been introduced into the story. When you Google "Schrodinger's cat" the following definition appears: "a cat imagined as being enclosed in a box with a radioactive source and a poison that will be released when the source (unpredictably) emits radiation, the cat being considered (according to quantum mechanics) to be simultaneously both dead and alive until the box is opened and the cat observed." The notion that a cat can be both alive and dead at the same time is counterinuitive and, most importantly, completely false. I propose a thought experiment that demonstrates that the popular conception of a half-living cat is impossible.

In [1]:
'''The following code is written in Python 3. The raw code is provided for transparency and reproducibility but does not need
to be read to understand the experiment.'''

#Importing some things that we'll need later
import random
import numpy as np
from functools import reduce
from IPython.core.debugger import set_trace

Let's assume we have the typical Schrödinger's cat experiment. We'll let the experiment run until the probability that the vial of poison is destroyed and therefore the cat is dead is 50%. After that time the vial will be removed and we will attempt to determine whether the cat is dead or alive.

Now let's set up the rest of the experiment. Attached to the experimental device is a printer. The printer knows whether the cat is alive or dead, but instead of just printing the results, it throws a little randomness into the mix. For its first printout, it will print either "The cat is alive" or "The cat is dead" with a 50% probability. In this case, the printout isn't in any way useful for determining the status of the cat. But for each print out after the first, the probability that the printout will be accurate increases by 1%. Thus the first will be accurate 50% of the time, the 11th printout 60% of the time, and the 26th will be accurate 75% of the time.

OK, now we'll got our experiment and our printer set up, let's run the experiment!

In [2]:
# Generate a list of random numbers to work with
# Generate numbers uniformly distributed between 0 and 1
np.random.seed(0)
# Generate 51 different numbers
randoms = np.random.rand(51)

# Create an array to hold the results
all_results = np.full(51, np.nan)

# Conduct the experiment
# simulates the Schrödinger's cat experiment by outputting either a 1, for alive, or a 0.
result = np.random.randint(0,2)

In [3]:
# Convert result to human readable
if result == 1:
    true_status = 'alive'
    false_status = 'dead'
else:
    true_status = 'dead'
    false_status = 'alive'

After running the experiment the vial is removed and the cat is either alive or dead, but we do not know. We'll print our first result,  which, because it's only accurate 50% of the time, will contain no actual information about the state of the cat. Here is the first printout:

In [4]:
random_number = randoms[0] # -1 to adjust for 0-index
if random_number<0.50: # 50% chance in the beginning
    all_results[0] = result
    print("The cat is {}\n".format(true_status))
else:
    all_results[0] = not result
    print("The cat is {}\n".format(false_status))

The cat is alive



In [5]:
def run_experiment(trials, observer, initial_trial=1):
    for current_trial_number in range(initial_trial, initial_trial + trials):
        #set_trace()    
        probability_of_truth = round(0.50 + (current_trial_number-1)/100,2) # Starts at 50% plus 1% for every previous trial and increases by 1% every iteration
        print(("Trial number {num}, where the printout has a {prob:.0%} chance of being accurate.\n{obser}'s printout says:".format(
                num=current_trial_number, prob=probability_of_truth, obser=observer)))
        if randoms[current_trial_number] < probability_of_truth: # If the printer prints the true outcome
            all_results[current_trial_number] = result
            if observer == 'Alice':
                print("    The cat is {}\n".format(true_status))
            elif observer == 'Bob':
                print("    That statement was true\n")
            elif observer == 'Charlie':
                print("    The cat is {}".format(true_status))
                print("    That statement was true\n")
        else: # if the printer lies about the outcome
            all_results[current_trial_number] = not result
            if observer == 'Alice':
                print("    The cat is {}\n".format(false_status))
            elif observer == 'Bob':
                print("    That statement was false\n")
            elif observer == 'Charlie':
                print("    The cat is {}".format(false_status))
                print("    That statement was false\n")

In [6]:
# Instead of the above, just do this:
run_experiment(1, 'Alice')

Trial number 1, where the printout has a 50% chance of being accurate.
Alice's printout says:
    The cat is alive



Now the experiment has been conducted and a result has been printed, although the result is meaningless. Now we'll start with the real trials. For the first trial, the prinout will be accurate 51% of the time. Thus the next printout will be the truth with a probability of 51% and a lie with a probability of 49%. And the one after that will have a 52% chance of being true, and so on.

In [7]:
run_experiment(5, 'Alice', 2)

Trial number 2, where the printout has a 51% chance of being accurate.
Alice's printout says:
    The cat is alive

Trial number 3, where the printout has a 52% chance of being accurate.
Alice's printout says:
    The cat is alive

Trial number 4, where the printout has a 53% chance of being accurate.
Alice's printout says:
    The cat is dead

Trial number 5, where the printout has a 54% chance of being accurate.
Alice's printout says:
    The cat is alive

Trial number 6, where the printout has a 55% chance of being accurate.
Alice's printout says:
    The cat is dead



If it so happens that the person receiving the printout, we'll call her Alice, is a statistician. With her background, she is able to quickly calculate the true probability, based on the information she has, that the cat is alive.

In [8]:
def calc_prob(printed_results):
    # Remove the values that are nan
    completed_results = printed_results[~np.isnan(printed_results)]
    prob_if_true = []
    prob_if_false = []
    for x in range(len(completed_results)):
        if printed_results[x] == 1:
            prob_if_true.append(50+x)
            prob_if_false.append(50-x)
        else: # if the value is 0
            prob_if_true.append(50-x)
            prob_if_false.append(50+x)
    # Multiply all the elements in the list together
    relative_positive = reduce(lambda x, y: x*y, prob_if_true)
    relative_negative = reduce(lambda x, y: x*y, prob_if_false)
    probability_alive = relative_positive / (relative_positive + relative_negative)
    print(("Alice calculates that the probability that the cat survived the experiment is {probability:.2%}".format(probability=probability_alive)))

In [9]:
calc_prob(all_results)

Alice calculates that the probability that the cat survived the experiment is 50.98%


Then we add a second person to the experiment. We'll call him Bob. Bob never sees or communicates with Alice, but he does get a printout from the experiment. However, instead of printing "The cat is alive" or "The cat is dead", it prints either "That statement was true" or "That statement was false". Now, Bob knows whether the statement that Alice is reading is true or false, but he does not know what Alice is acutally reading, and therefore doesn't know anything about the cat.

Let's continue with trials 11-20. But instead of seeing what Alice sees, we'll see only what Bob sees.

In [10]:
run_experiment(5, 'Bob', 6)

Trial number 6, where the printout has a 55% chance of being accurate.
Bob's printout says:
    That statement was true

Trial number 7, where the printout has a 56% chance of being accurate.
Bob's printout says:
    That statement was false

Trial number 8, where the printout has a 57% chance of being accurate.
Bob's printout says:
    That statement was false

Trial number 9, where the printout has a 58% chance of being accurate.
Bob's printout says:
    That statement was true

Trial number 10, where the printout has a 59% chance of being accurate.
Bob's printout says:
    That statement was false



We see that Alice got five false statements and five true ones, but we don't know what statements we got. Based on what he's seeing, Bob can't determine anything about the result of the experiment. Now let's go back to Alice and run some more trials.

In [11]:
run_experiment(10, 'Alice', 11)

Trial number 11, where the printout has a 60% chance of being accurate.
Alice's printout says:
    The cat is dead

Trial number 12, where the printout has a 61% chance of being accurate.
Alice's printout says:
    The cat is dead

Trial number 13, where the printout has a 62% chance of being accurate.
Alice's printout says:
    The cat is alive

Trial number 14, where the printout has a 63% chance of being accurate.
Alice's printout says:
    The cat is dead

Trial number 15, where the printout has a 64% chance of being accurate.
Alice's printout says:
    The cat is dead

Trial number 16, where the printout has a 65% chance of being accurate.
Alice's printout says:
    The cat is dead

Trial number 17, where the printout has a 66% chance of being accurate.
Alice's printout says:
    The cat is alive

Trial number 18, where the printout has a 67% chance of being accurate.
Alice's printout says:
    The cat is alive

Trial number 19, where the printout has a 68% chance of being accurat

All the while Bob is in his room getting his printouts. There is still a lot of noise in her data. But she can calculate the probability again. She'll include the results she got when we were in the room with Bob and we didn't see. This time she gets:

In [12]:
all_results

array([  1.,   1.,   1.,   1.,   0.,   1.,   0.,   1.,   1.,   0.,   1.,
         0.,   0.,   1.,   0.,   0.,   0.,   1.,   1.,   1.,   1.,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan])

In [13]:
calc_prob(all_results)

Alice calculates that the probability that the cat survived the experiment is 82.24%


OK, now Alice is feeling more confident that she knows the answer. Let's add another person, Charlie, to the experiment. Charlie doesn't get any printout, but he can talk to both Alice and Bob. They run another trial but now Alice and Bob tell their results to Charlie. Let's take a look at what Charlie sees.

In [14]:
run_experiment(5, 'Charlie', 21)

Trial number 21, where the printout has a 70% chance of being accurate.
Charlie's printout says:
    The cat is alive
    That statement was false

Trial number 22, where the printout has a 71% chance of being accurate.
Charlie's printout says:
    The cat is dead
    That statement was true

Trial number 23, where the printout has a 72% chance of being accurate.
Charlie's printout says:
    The cat is alive
    That statement was false

Trial number 24, where the printout has a 73% chance of being accurate.
Charlie's printout says:
    The cat is dead
    That statement was true

Trial number 25, where the printout has a 74% chance of being accurate.
Charlie's printout says:
    The cat is dead
    That statement was true



For the final step of the experiment, Bob goes into the room with the experiment and opens the box. He looks for the first time to see the ultimate fate of the cat.
This concludes the experiment.

According to the definition provided by Google, the cat remains in a state of being both alive and dead until the box is opened and the cat is observed. In this experiment, there are many points where the question of whether the radioactive material has decayed could be said to be answered. The following are test points when the quantum phenomenon could be said to be resolved:

- The machine prints the 50-50 result to Alice.

- The machine prints the 51-49 result to Alice, marking the first time the experiment affected the macroscopic world.

- Alice calculates the probability the cat is alive, marking the first time a person has a good estimate of the probability.

- Bob enters the experiment and the result of the experiment could be determined by knowing what both Alice and Bob know.

- Charlie enters the experiment and become the first person to know the result of the experiment with certainty.

- Bob opens the container and obeserves the cat directly.

This thought experiment shows that none of the above explanations are satisfactory, and the only possible explanation is that the observation occurs at every point in time, when the first macroscopic entity, the Geiger counter, either detects or does not detect the radiation which causes it to release the hammer smashing the vial of poison. The presence of humans or human consciousness has no effect on quantum mechanical systems.

Go [here](https://nbviewer.jupyter.org/github/jss367/JupyterNotebooks/blob/master/Schr%C3%B6dinger%27s%20cat.ipynb) to see the code for this post.